In [2]:
import os
import pandas as pd
import numpy as np
import cv2

# Translation
def translate(data, shift, direction='right'):
    """
    Translate the data in the specified direction.
    :param data: NumPy array
    :param shift: Integer, the number of steps to shift
    :param direction: String, direction of the shift ('right', 'left', 'down', 'up')
    :return: Translated NumPy array
    """
    if direction == 'right':
        return np.roll(data, shift, axis=1)
    elif direction == 'left':
        return np.roll(data, -shift, axis=1)
    elif direction == 'down':
        return np.roll(data, shift, axis=0)
    elif direction == 'up':
        return np.roll(data, -shift, axis=0)
    else:
        raise ValueError("Direction must be 'right', 'left', 'down', or 'up'.")


# Chemin du dossier parent contenant les sous-dossiers avec fichiers VTK et RES
dossier_parent = '/Users/Bader/Desktop/Mines 2A/Projet 2A/CODES/donnees/Database/results'

# Parcourir les sous-dossiers
for sous_dossier in os.listdir(dossier_parent):
    chemin_sous_dossier = os.path.join(dossier_parent, sous_dossier)

    # Vérifier si le chemin pointe vers un dossier (plutôt qu'un fichier)
    if os.path.isdir(chemin_sous_dossier):
        # Liste des fichiers dans le sous-dossier
        fichiers_sous_dossier = os.listdir(chemin_sous_dossier)

        # Filtrage réduits :
        fichiers_a_comp = [fichier for fichier in fichiers_sous_dossier if fichier.endswith("reduced.parquet")]

        # Vérification qu'il y a au moins un fichier vtk et un fichier .res
        if fichiers_a_comp :
            # Extraction des noms de fichiers
            # nom_du_dossier = os.path.basename(chemin_sous_dossier)

            # Prétraiter le fichier VTK
            tableau_init = pd.read_parquet(chemin_sous_dossier + '/' + fichiers_a_comp[0])
            n = tableau_init.shape[0]
            tableau_init_np = tableau_init.to_numpy()


            L_new_tab = [] # Liste pour nouvelles données
            dir = ['right', 'up', 'left', 'down',]

            # Translations
            rand = [np.random.randint(0,n) for k in range(4)]
            for i in range(len(dir)) :
                L_new_tab.append(translate(tableau_init_np, rand[i], dir[i]))

            # Rotations
            L_new_tab.append(np.rot90(tableau_init_np, k=1))
            L_new_tab.append(np.rot90(tableau_init_np, k=-1))

            L_new_tab = [pd.DataFrame(tab) for tab in L_new_tab]
            for df in L_new_tab :
                df.columns = [str(col) for col in df.columns]

            for k in range(len(L_new_tab)):
                nom_fichier_parquet = fichiers_a_comp[0].split(".")[0]+ '_{}augred'.format(k) + '.parquet'


                #Définition des chemins pour la sauvegarde des fichiers
                chemin_sauvegarde_parquet = os.path.join(chemin_sous_dossier, nom_fichier_parquet)
                #Sauuvegarde au format parquet
                L_new_tab[k].to_parquet(chemin_sauvegarde_parquet)